In [1]:
#Instalar las librerías de uso
!pip install langchain
!pip install cohere
!pip install unstructured
!pip install "unstructured[pdf]"
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.3 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 6.0 MB/s eta 0:00:00


In [2]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 0s (975 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 126308 

In [3]:
# Instalar la extensión de langchain para el chatbot con Cohere

!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00


In [4]:
#Importar los módulos, particularmente los de pregunta-respuesta de Cohere

from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.retrievers import TFIDFRetriever
from langchain.memory import ConversationBufferMemory


In [5]:
# Cargar el documento de la carpeta docs (contenido). Se carga el Guión (Guión_HMI)

from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("/content/docs")
docs = loader.load()

In [6]:
# Aplicar Split sobre el texto para separarlo en badges

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)
len(texts)

142

In [7]:
# Uso de la API KEY de Cohere para el chatbot (no cambiar)

cohere_api_key='1msKL9N3DxmNqmxMCQLQ4CHz8e1dO130v1urBoUI'

# Crear el embedding con el modelo libre "multilingual-22-12" de Cohere para su uso en español. No está en uso pero hizo parte de las pruebas.

embeddings = CohereEmbeddings(
        model="multilingual-22-12", cohere_api_key=cohere_api_key,user_agent="my-app"
    )

/tmp/ipython-input-7-3768990833.py:7: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereEmbeddings``.
  embeddings = CohereEmbeddings(


In [8]:
# Creación del contexto para el chatbot

#db = Qdrant.from_documents(texts, embeddings, location=":memory:", collection_name="test", distance_func="Dot")

db = TFIDFRetriever.from_documents(texts)

### Chatbot

In [9]:
#Configuración del Prompt (En inglés para mayor precisión). Incluye instrucciones, contexto (Guión), pregunta (usuario) e historial de chat.

prompt_template = """

## Instructions
You are an AI personal friendly assistant named Scientia, designed to answer questions about the radar of the IA.
You MUST only support Spanish for questions and answers. Your responses should be concise and directly address the specific question.
Answer based solely on the content of the provided documents. Do not generate an answer that is not supported by the documents.
If you cannot find the answer to the user's question in the documents provided, respond by stating that the information is beyond your scope.

Use the following documents and chat history to answer the question in Spanish:

Question:{question}

Documents: {context}

Chat History: {history}

"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question", "history"]
)

In [10]:
# Creación del chatbot. El modelo usado es "command-r-08-2024" por tener las mejores cualidades en español. Incluye el Prompt creado.

qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-r-08-2024", temperature=0.75,cohere_api_key=cohere_api_key,max_tokens=300),
                                 chain_type="stuff",
                                 retriever=db,
                                 verbose=False,
                                 chain_type_kwargs = {"verbose": False,"prompt": PROMPT,
                                                      "memory": ConversationBufferMemory(
                                                                memory_key="history",
                                                                input_key="question"),})

/tmp/ipython-input-10-1591314967.py:3: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  qa = RetrievalQA.from_chain_type(llm=Cohere(model="command-r-08-2024", temperature=0.75,cohere_api_key=cohere_api_key,max_tokens=300),
/tmp/ipython-input-10-1591314967.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  "memory": ConversationBufferMemory(


In [11]:
#Prueba de respuesta

answer = qa.invoke({"query": "¿Qué es el radar de la IA?"})
print(answer)

{'query': '¿Qué es el radar de la IA?', 'result': 'El radar de la IA es un marco conceptual que permite evaluar y guiar el desarrollo de aplicaciones de inteligencia artificial de manera ética y responsable. Este radar considera cinco ejes fundamentales: diseño ético, alfabetización en IA, uso ético, manejo de riesgos y regulación. Cada eje se analiza a través de una serie de preguntas orientadoras y las aplicaciones reciben puntuaciones numéricas y observaciones cualitativas. El índice del radar se calcula a partir de estas puntuaciones y proporciona una herramienta práctica para comparar y mejorar el desarrollo de la IA.'}
